# Create a voronoi for each FSP maps type

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from iso3166 import countries
import matplotlib.pyplot as plt
%matplotlib inline 
# The shapely.ops module has a cascaded_union that finds the cumulative union of many objects
from shapely.ops import cascaded_union

### Read country maps

In [2]:
iso = ['BGD','IND','KEN','LSO','NGA','TZA','UGA']

In [3]:
# Bangladesh
bgdMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_BGD.shp')
# India
indMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_IND.shp')
# Get Uttar Pradesh and Bihar states
indMapStates = indMap[(indMap['NAME_1'] == 'Uttar Pradesh') | (indMap['NAME_1'] == 'Bihar')]
# Kenya
kenMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_KEN.shp')
# Lesotho
lsoMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_LSO.shp')
# Nigeria
ngaMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_NGA.shp')
# Tanzania
tzaMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_TZA.shp')
# Uganda
ugaMap = gpd.read_file('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/gadm36_shp/gadm36_UGA.shp')

Get the boundary of each country

In [159]:
bgdBoundary = gpd.GeoSeries(cascaded_union(bgdMap['geometry']))
bgdBoundary = gpd.GeoDataFrame(bgdBoundary).rename(columns={0: 'geometry'})
bgdBoundary['country'] = 'Bangladesh'
bgdBoundary['iso'] = 'BGD'

indBoundary = gpd.GeoSeries(cascaded_union(indMapStates['geometry']))
indBoundary = gpd.GeoDataFrame(indBoundary).rename(columns={0: 'geometry'})
indBoundary['country'] = 'India'
indBoundary['iso'] = 'IND'

kenBoundary = gpd.GeoSeries(cascaded_union(kenMap['geometry']))
kenBoundary = gpd.GeoDataFrame(kenBoundary).rename(columns={0: 'geometry'})
kenBoundary['iso'] = 'KEN'

lsoBoundary = gpd.GeoSeries(cascaded_union(lsoMap['geometry']))
lsoBoundary = gpd.GeoDataFrame(lsoBoundary).rename(columns={0: 'geometry'})
lsoBoundary['country'] = 'Lesotho'
lsoBoundary['iso'] = 'LSO'

ngaBoundary = gpd.GeoSeries(cascaded_union(ngaMap['geometry']))
ngaBoundary = gpd.GeoDataFrame(ngaBoundary).rename(columns={0: 'geometry'})
ngaBoundary['country'] = 'Nigeria'
ngaBoundary['iso'] = 'NGA'

tzaBoundary = gpd.GeoSeries(cascaded_union(tzaMap['geometry']))
tzaBoundary = gpd.GeoDataFrame(tzaBoundary).rename(columns={0: 'geometry'})
tzaBoundary['country'] = 'Tanzania'
tzaBoundary['iso'] = 'TZA'

ugaBoundary = gpd.GeoSeries(cascaded_union(ugaMap['geometry']))
ugaBoundary = gpd.GeoDataFrame(ugaBoundary).rename(columns={0: 'geometry'})
ugaBoundary['country'] = 'Uganda'
ugaBoundary['iso'] = 'UGA'

boundaries = gpd.GeoDataFrame(pd.concat([bgdBoundary,indBoundary,kenBoundary,lsoBoundary,ngaBoundary,tzaBoundary,ugaBoundary]))

### Voronoi tessellation finite_polygons
Built a Voronoi tessellation from points

In [5]:
from scipy.spatial import Voronoi
from shapely.geometry import Polygon, Point

In [6]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.

    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.

    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.

    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [7]:
def voronoi_tesellation_box(boundary,lng,lat):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    boundary : GeoDataFrame, 
        Geometry of the country.
    lng : GeoSeries, 
        Longitud values of points. 
    lat : GeoSeries, 
        Longitud values of points. 
    Returns
    -------
    voronoid : GeaoDataFrames
        Geometries of Voronoi regions.
    """
    # array with points coordinates
    points = np.zeros((lng.shape[0],2))
    points[:,0] = lng
    points[:,1] = lat

    # compute Voronoi tesselation
    vor = Voronoi(points)
    
    # Reconstruct infinite voronoi regions in a 2D diagram to finite regions.
    regions, vertices = voronoi_finite_polygons_2d(vor)
    
    # build box from country boundary
    xmin = boundary.bounds.minx[0]
    xmax = boundary.bounds.maxx[0]
    ymin = boundary.bounds.miny[0]
    ymax = boundary.bounds.maxy[0]

    box = Polygon([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])

    voronoid = [] 
    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        voronoid.append(poly.intersection(box))
        
    voronoid = gpd.GeoDataFrame(geometry = voronoid)
    
    vor_lng = vor.points[:,0]
    vor_lat = vor.points[:,1]
    
    voronoid['lng'] = vor_lng
    voronoid['lat'] = vor_lat
    
    return voronoid    

Intersect voronoid with boundary

In [8]:
def spatial_overlays(df1, df2):
    '''Compute overlay intersection of two 
        GeoPandasDataFrames df1 and df2
    '''
    df1 = df1.copy()
    df2 = df2.copy()
    df1['geometry'] = df1.geometry.buffer(0)
    df2['geometry'] = df2.geometry.buffer(0)

    # Spatial Index to create intersections
    spatial_index = df2.sindex
    df1['bbox'] = df1.geometry.apply(lambda x: x.bounds)
    df1['histreg']=df1.bbox.apply(lambda x:list(spatial_index.intersection(x)))
    pairs = df1['histreg'].to_dict()
    nei = []
    for i,j in pairs.items():
        for k in j:
            nei.append([i,k])
        
    pairs = gpd.GeoDataFrame(nei, columns=['idx1','idx2'], crs=df1.crs)
    pairs = pairs.merge(df1, left_on='idx1', right_index=True)
    pairs = pairs.merge(df2, left_on='idx2', right_index=True, suffixes=['_1','_2'])
    pairs['Intersection'] = pairs.apply(lambda x: (x['geometry_1'].intersection(x['geometry_2'])).buffer(0), axis=1)
    pairs = gpd.GeoDataFrame(pairs, columns=pairs.columns, crs=df1.crs)
    cols = pairs.columns.tolist()
    cols.remove('geometry_1')
    cols.remove('geometry_2')
    cols.remove('histreg')
    cols.remove('bbox')
    cols.remove('Intersection')
    dfinter = pairs[cols+['Intersection']].copy()
    dfinter.rename(columns={'Intersection':'geometry'}, inplace=True)
    dfinter = gpd.GeoDataFrame(dfinter, columns=dfinter.columns, crs=pairs.crs)
    dfinter = dfinter.loc[dfinter.geometry.is_empty==False]
    dfinter.drop(['idx1','idx2'], axis=1, inplace=True)
    return dfinter

### FSP maps voronoid table

In [153]:
# Read table
df = pd.read_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/FSP_Maps/FSP_maps.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [222]:
df_vor = pd.DataFrame(columns=['geometry','lng','lat','iso','sector','type','type_id'])
for country in iso:
    boundary = gpd.GeoDataFrame(boundaries[boundaries['iso'] == country]['geometry'])
    df_maps = df[df['iso'] == country]
    for type in df_maps['type'].unique():
        #print('iso: ', country)
        #print('type: ', type)
        lng = df_maps[df_maps['type'] == type]['lng']
        lat = df_maps[df_maps['type'] == type]['lat']
        
        #exceptions
        if (country == 'IND' and type == 'Bank Customer Service Points'):
            lat.iloc[9265] = "{0:.4f}".format(lat.iloc[9265])
        if (country == 'LSO' and type == 'Pos Terminals'):
            lat.iloc[32] = "{0:.4f}".format(lat.iloc[32])
        
        #we need at least 4 points 
        if len(lat) >= 4:   
            voronoid = voronoi_tesellation_box(boundary,lng,lat)
            voronoid['iso'] = country
            voronoid['sector'] = df_maps[df_maps['type'] == type]['sector'].iloc[0]
            voronoid['type'] = type 
            voronoid['type_id'] = df_maps[df_maps['type'] == type]['type_id'].iloc[0]
    
            # Coordinate reference system : WGS84
            voronoid.crs = {'init': 'epsg:4326'}
        else:
            voronoid = pd.DataFrame(columns=['geometry','lng','lat','sector','type','type_id'])
            voronoid['lng'] = lng
            voronoid['lat'] = lat
            voronoid['geometry'] = ''
            voronoid['iso'] = country
            voronoid['sector'] = df_maps[df_maps['type'] == type]['sector'].iloc[0]
            voronoid['type'] = type 
            voronoid['type_id'] = df_maps[df_maps['type'] == type]['type_id'].iloc[0]              
    
        df_vor = pd.concat([df_vor,voronoid])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


**Add id column**

In [223]:
df_vor.reset_index(drop=True, inplace=True)
df_vor['id'] = np.arange(len(df_vor))
df_vor = df_vor[['id', 'geometry', 'iso', 'sector', 'type', 'type_id']]
df_vor.head()

,id,geometry,iso,sector,type,type_id
0,0,"POLYGON ((90.39619324657353 23.71580798075595,...",BGD,Finance,Bank Branches,1
1,1,"POLYGON ((91.66436500000006 25.03444576923081,...",BGD,Finance,Bank Branches,1
2,2,"POLYGON ((89.02918366984161 24.73270970308704,...",BGD,Finance,Bank Branches,1
3,3,"POLYGON ((88.59068851891206 25.12553230981727,...",BGD,Finance,Bank Branches,1
4,4,"POLYGON ((91.0043668141593 23.48641918141595, ...",BGD,Finance,Bank Branches,1


**Save table**

In [224]:
df_vor.to_csv('/Users/ikersanchez/Vizzuality/PROIEKTUAK/i2i/Data/FSP_Maps/FSP_voronoid.csv')

**Add area_km2 column in carto**

```sql
SELECT *, ST_Area(ST_Transform(the_geom,26986))*1e-6 As area_km2
FROM fsp_voronoid
ORDER BY id
```